# Bulbapedia

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

URL="https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_National_Pok%C3%A9dex_number"
page=requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
poke_content=soup.find(id='mw-content-text')

poke_tables=poke_content.find_all('table')
poke_headers = poke_content.find_all('h3')

In [3]:
table_start = 1
info_start = 3

poke_json = {}

for i in range(table_start, table_start + 8):
  temp_list = []
  for j in tqdm(range(info_start, len(poke_tables[i].contents), 2)):
    poke_info = poke_tables[i].contents[j]
    kdex = poke_info.contents[1].text.strip()
    ndex = poke_info.contents[3].text.strip()
    name = poke_info.contents[7].text.strip()
    URL = 'https://bulbapedia.bulbagarden.net' + poke_info.contents[7].a.get('href')
    types = []
    types.append(poke_info.contents[9].text.strip())
    if len(poke_info.contents) > 10:
        types.append(poke_info.contents[11].text.strip())
    temp_list.append({
          "kdex": kdex,
          "ndex": ndex,
          "name": name,
          "types": types,
          "URL": URL
      })
  poke_json[poke_headers[i - table_start].span.get('id')] = temp_list

100%|██████████| 94/94 [00:00<00:00, 18886.93it/s]


In [21]:
import json
with open('pokemon.json', 'w') as f:
    json.dump(poke_json, f, indent = 4)

# News Site

In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

URL=["https://www.dailymail.co.uk/home/sitemaparchive/day_20210311.html", "https://www.dailymail.co.uk/home/sitemaparchive/day_20210312.html"]

In [16]:
news_json = []

for url in URL:
  page=requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')

  articles = soup.find('ul', class_='archive-articles debate link-box').contents
  articles_start = 1

  # due to insufficient memory when getting the complete data set, only a 4th of the data will be retrieved
  for i in tqdm(range(articles_start, len(articles)//4, 2)):
    article = articles[i]
    article_url = 'https://www.dailymail.co.uk' + article.a.get('href')
    article_page=requests.get(article_url)
    article_soup = BeautifulSoup(article_page.content, 'html.parser')
    article_text = article_soup.find(id='js-article-text')

    date = article_text.find('span', class_='article-timestamp article-timestamp-published').time.get('datetime')[:10].strip()
    if date != '2021-03-12' and date != '2021-03-11':
      continue
    
    try:
      title = article_text.h2.text.strip()
    except:
      title = article_text.h1.text.strip()
    
    full_text = article_text.find('div', itemprop='articleBody').text.strip()
    
    try:
      author_a = article_text.find_all('a', class_='author')
      author = [x.text.strip() for x in author_a]
    except:
      author = []

    news_json.append({
        "date": date,
        "title": title,
        "full article": full_text,
        "author": author
    })

100%|██████████| 500/500 [04:51<00:00,  1.71it/s]


In [20]:
import json
with open('news_site.json', 'w') as f:
    json.dump(news_json, f, indent = 4)